In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sb

In [ ]:
import PIL
import cv2
import tensorflow as tf
from tensorflow import keras

In [ ]:
import pathlib

In [ ]:
from google.colab import drive

In [ ]:
drive.mount("/content/drive")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
url="/content/drive/MyDrive/tomato/train"

In [ ]:
url=pathlib.Path(url)

In [ ]:
data={"Tomato_Bacterial_spot":list(url.glob("Tomato___Bacterial_spot/*"))[:400],
      "Tomato_Early_blight":list(url.glob("Tomato___Early_blight/*"))[:400],
      "Tomato_Late_blight":list(url.glob("Tomato___Late_blight/*"))[:400],
      "Tomato_Leaf_Mold":list(url.glob("Tomato___Leaf_Mold/*"))[:400],
      "Tomato_Septoria_leaf_spot":list(url.glob("Tomato___Septoria_leaf_spot/*"))[:400],
      "Tomato_Spider_mites Two-spotted_spider_mite":list(url.glob("Tomato___Spider_mites Two-spotted_spider_mite/*"))[:400],
      "Tomato_Target_Spot":list(url.glob("Tomato___Target_Spot/*"))[:400],
      "Tomato_Tomato_Yellow_Leaf_Curl_Virus":list(url.glob("Tomato___Tomato_Yellow_Leaf_Curl_Virus/*"))[:400],
      "Tomato_Tomato_mosaic_virus":list(url.glob("Tomato___Tomato_mosaic_virus/*"))[:400],
      "Tomato_healthy":list(url.glob("Tomato___healthy/*"))[:400]
      }

In [ ]:
len(data["Tomato_Bacterial_spot"])

400

In [ ]:
index={"Tomato_Bacterial_spot":1,
      "Tomato_Early_blight":2,
      "Tomato_Late_blight":3,
      "Tomato_Leaf_Mold":4,
      "Tomato_Septoria_leaf_spot":5,
      "Tomato_Spider_mites Two-spotted_spider_mite":6,
      "Tomato_Target_Spot":7,
      "Tomato_Tomato_Yellow_Leaf_Curl_Virus":8,
      "Tomato_Tomato_mosaic_virus":9,
      "Tomato_healthy":0
      }

In [ ]:
x,y=[],[]
for nam,li in data.items():
  for l in li:
    img_arr=cv2.imread(str(l))
    img_res=cv2.resize(img_arr,(224,224))
    x.append(img_res)
    y.append(index[nam])

In [ ]:
x=np.array(x)
y=np.array(y)

In [ ]:
x=x/255

In [ ]:
x.shape

(4000, 224, 224, 3)

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.3)

In [ ]:
x_train.shape

(2800, 224, 224, 3)

In [ ]:
import tensorflow_hub as hub

In [ ]:
data_augmentation=keras.Sequential([
      keras.layers.experimental.preprocessing.RandomZoom(0.4),
      keras.layers.experimental.preprocessing.RandomContrast(0.3),
      keras.layers.experimental.preprocessing.RandomRotation(0.5)
])

In [ ]:
model=keras.Sequential([
      data_augmentation,
      hub.KerasLayer("https://tfhub.dev/tensorflow/resnet_50/feature_vector/1",input_shape=(224,224,3),trainable=False),
      keras.layers.Dense(10,activation="softmax")
])
model.compile(
    optimizer="adam",
    loss="sparse_categorical_crossentropy",
    metrics=["accuracy"]
)
model.fit(x_train,y_train,epochs=25)

Epoch 1/25
88/88 [==============================] - 26s 219ms/step - loss: 1056.5724 - accuracy: 0.1050
Epoch 2/25
88/88 [==============================] - 19s 219ms/step - loss: 322.7875 - accuracy: 0.1832
Epoch 3/25
88/88 [==============================] - 19s 219ms/step - loss: 381.2628 - accuracy: 0.1932
Epoch 4/25
88/88 [==============================] - 19s 218ms/step - loss: 282.4228 - accuracy: 0.2582
Epoch 5/25
88/88 [==============================] - 19s 219ms/step - loss: 342.7203 - accuracy: 0.2532
Epoch 6/25
88/88 [==============================] - 19s 218ms/step - loss: 223.5256 - accuracy: 0.3293
Epoch 7/25
88/88 [==============================] - 19s 219ms/step - loss: 196.9765 - accuracy: 0.3179
Epoch 8/25
88/88 [==============================] - 19s 218ms/step - loss: 245.0264 - accuracy: 0.3282
Epoch 9/25
88/88 [==============================] - 19s 218ms/step - loss: 289.0401 - accuracy: 0.2993
Epoch 10/25
88/88 [==============================] - 19s 218ms/step - lo

In [ ]:
model.evaluate(x_test,y_test)

38/38 [==============================] - 9s 216ms/step - loss: 280.6782 - accuracy: 0.3833


[280.6781921386719, 0.38333332538604736]